In [2]:
import pandas as pd


In [3]:
df = pd.read_csv("merged_output.csv")
sampled_df = df.sample(n=5000, random_state=42)

In [15]:
df.shape

(66074219, 5)

In [13]:
df.head()

,transition_date,transition_hour,line,number_of_passenger,station_poi_desc_cd
0,2023-01-01,0,USKUDAR-CEKMEKOY,1,ALTUNIZADE 1
1,2023-01-01,0,USKUDAR-CEKMEKOY,296,USKUDAR 1
2,2023-01-01,0,KADIKOY-KARTAL,14,GOZTEPE
3,2023-01-01,0,KADIKOY-KARTAL,5,GOZTEPE
4,2023-01-01,0,USKUDAR-CEKMEKOY,1,CAKMAK


In [4]:
sampled_df['timestamp'] = pd.to_datetime(sampled_df['transition_date']) + pd.to_timedelta(sampled_df['transition_hour'], unit='h')


In [14]:
from kafka import KafkaProducer
import json
import pandas as pd
import time

# Initialize Kafka producer
producer = KafkaProducer(
    bootstrap_servers='192.168.113.33:9092',
    value_serializer=lambda x: json.dumps(
        {k: (v.isoformat() if isinstance(v, pd.Timestamp) else v) for k, v in x.items()}
    ).encode('utf-8')
)

for index, row in sampled_df.iterrows():
    print(f"Streaming data for line: {index}")
    
    message = row.to_dict()
    
    producer.send("publictransportstream", key=message["line"].encode('utf-8'), value=message)
    print(f"Sent message: {message}")
    
    time.sleep(0.001)

Streaming data for line: 65964506
Sent message: {'transition_date': '2023-12-31', 'transition_hour': 12, 'line': 'BOSTANCI - DUDULLU', 'number_of_passenger': 4, 'station_poi_desc_cd': 'BOSTANCI', 'timestamp': Timestamp('2023-12-31 12:00:00')}
Streaming data for line: 14373845
Sent message: {'transition_date': '2023-03-28', 'transition_hour': 19, 'line': 'YENIKAPI - HACIOSMAN', 'number_of_passenger': 5, 'station_poi_desc_cd': 'SISHANE GUNEY', 'timestamp': Timestamp('2023-03-28 19:00:00')}
Streaming data for line: 62839520
Sent message: {'transition_date': '2023-12-14', 'transition_hour': 15, 'line': 'KADIKOY-KARTAL', 'number_of_passenger': 118, 'station_poi_desc_cd': 'SOGANLIK', 'timestamp': Timestamp('2023-12-14 15:00:00')}
Streaming data for line: 10198951
Sent message: {'transition_date': '2023-03-03', 'transition_hour': 20, 'line': 'KADIKOY-KARTAL', 'number_of_passenger': 3, 'station_poi_desc_cd': 'KADIKOY (DOGU)', 'timestamp': Timestamp('2023-03-03 20:00:00')}
Streaming data for li